Following tutorial described here: https://docs.prefect.io/tutorials/first-steps/

In [2]:
from prefect import flow

In [3]:
@flow
def my_favorite_function():
    print("What is your favorite number?")
    return 42

In [4]:
print(my_favorite_function())

02:40:41.707 | INFO    | prefect.engine - Created flow run 'quantum-buffalo' for flow 'my-favorite-function'
02:40:42.124 | INFO    | Flow run 'quantum-buffalo' - Finished in state Completed()


What is your favorite number?
42


### Tasks are called from flows

In [5]:
import requests
from prefect import flow, task

@task
def call_api(url):
    response = requests.get(url)
    print(response.status_code)
    return response.json()

@task
def parse_fact(response):
    fact = response["fact"]
    print(fact)
    return fact

@flow
def api_flow(url):
    fact_json = call_api(url)
    fact_text = parse_fact(fact_json)
    return fact_text

api_flow("https://catfact.ninja/fact")

02:49:57.922 | INFO    | prefect.engine - Created flow run 'intelligent-kingfisher' for flow 'api-flow'
02:49:58.394 | INFO    | Flow run 'intelligent-kingfisher' - Created task run 'call_api-ded10bed-0' for task 'call_api'
02:49:58.394 | INFO    | Flow run 'intelligent-kingfisher' - Executing 'call_api-ded10bed-0' immediately...
02:49:59.519 | INFO    | Task run 'call_api-ded10bed-0' - Finished in state Completed()
02:49:59.608 | INFO    | Flow run 'intelligent-kingfisher' - Created task run 'parse_fact-6803447a-0' for task 'parse_fact'
02:49:59.608 | INFO    | Flow run 'intelligent-kingfisher' - Executing 'parse_fact-6803447a-0' immediately...


200


02:49:59.807 | INFO    | Task run 'parse_fact-6803447a-0' - Finished in state Completed()
02:49:59.879 | INFO    | Flow run 'intelligent-kingfisher' - Finished in state Completed()


Blue-eyed, white cats are often prone to deafness.


'Blue-eyed, white cats are often prone to deafness.'